In [ ]:
!python --version

In [ ]:
!pwd

In [ ]:
CONFIG = 'basic_rnn'
BUNDLE_PATH = '/root/magenta/magenta/models/melody_rnn/mag/basic_rnn.mag'
OUTPUT_DIR = '/root/magenta/magenta/models/melody_rnn/output/2020.3.7-01'

# 用 pre-trained model 生成序列

In [ ]:
!melody_rnn_generate \
--config='attention_rnn' \
--bundle_file='/root/magenta/magenta/models/melody_rnn/mag/attention_rnn.mag' \
--output_dir='/root/magenta/magenta/models/melody_rnn/output/2020.3.30' \
--num_outputs=5 \
--num_steps=256 \
--primer_midi='/root/magenta/magenta/models/melody_rnn/primer.mid'

# 制作数据集

**注：这一节的脚本只是记录下来的，由于输出过多，VS Code容易与服务器断开连接，所以脚本需要在终端运行。**

## Building your Dataset

In [ ]:
INPUT_DIRECTORY=/download/clean_midi

# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD=/root/notesequences.tfrecord

convert_dir_to_note_sequences \
  --input_dir=$INPUT_DIRECTORY \
  --output_file=$SEQUENCES_TFRECORD \
  --recursive

## Create SequenceExamples

In [ ]:
melody_rnn_create_dataset \
--config=attention_rnn \
--input=/root/notesequences.tfrecord \
--output_dir=/download/melody_rnn/sequence_examples \
--eval_ratio=0.10

# 训练模型

### 第一次训练(3.30-3.31): run1

In [ ]:
melody_rnn_train \
--config=attention_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run1 \
--sequence_example_file=/download/melody_rnn/sequence_examples/training_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--num_training_steps=20000

run1因为未知原因被中止。生成旋律：

In [ ]:
!melody_rnn_generate \
--config=attention_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run1 \
--output_dir=/root/magenta/magenta/models/melody_rnn/generated/run1 \
--num_outputs=10 \
--num_steps=128 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--primer_melody="[60]"

## 生成bundle file(.mag文件)

In [ ]:
!melody_rnn_generate \
--config=attention_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run1 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--bundle_file=/root/magenta/magenta/models/melody_rnn/generated/run1/attention_rnn.mag \
--save_generator_bundle